In [ ]:
# 初回のみ
#!wget -P '/content/drive/My Drive/' http://mdcrosby.com/builds/AnimalAI_LINUX_3.0.zip
#!unzip '/content/drive/MyDrive/AnimalAI_LINUX_3.0.zip' -d "/content/drive/MyDrive/最終課題/animal-ai/"
#!mv /content/drive/MyDrive/最終課題/animal-ai/AnimalAI_LINUX_3.0/* /content/drive/MyDrive/最終課題/animal-ai/env/

- tensorboard
```
tensorboard --logdir /content/drive/MyDrive/最終課題/animal-ai/examples/dqn_tensorboard
```

ls: cannot open directory '.': Transport endpoint is not connected


In [1]:
from google.colab import drive
drive.mount("./drive")
%cd "/content/drive/My Drive/最終課題/animal-ai"
!pip install -e animalai 1> /dev/null 
!pip install stable_baselines3 1> /dev/null
!pip install gym_unity 1> /dev/null
!apt update 1> /dev/null
!apt install xvfb 1> /dev/null
!pip install stable-baselines 1> /dev/null

Mounted at ./drive
/content/drive/My Drive/最終課題/animal-ai
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.8.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 1.5.0 requires gym==0.21, but you have gym 0.20.0 which is incompatible.
mlagents 0.27.0 requires mlagents-envs==0.27.0, but you have mlagents-envs 0.28.0 which is incompatible.






In [2]:
!chmod +x /content/drive/MyDrive/animal-ai/env/UnityPlayer.so
!chmod -R 755 /content/drive/MyDrive/最終課題/animal-ai/env/AnimalAI.x86_64

In [ ]:
from stable_baselines3 import DQN
from stable_baselines.bench import Monitor
import torch as th

import sys
import random
import os
# from stable_baselines3.common.vec_env.dummy_vec_env import DummyVecEnv
from gym_unity.envs import UnityToGymWrapper
from animalai.animalai.envs.environment import AnimalAIEnvironment

In [ ]:
!python3 dreamer/main.py

Streaming output truncated to the last 5000 lines.
update_step:  30 model loss: 3.00125, kl_loss: 3.00000, obs_loss: 0.00125, reward_loss: 0.00000, value_loss: 0.00002 action_loss: 0.95475
update_step:  40 model loss: 3.00105, kl_loss: 3.00000, obs_loss: 0.00105, reward_loss: 0.00000, value_loss: 0.00002 action_loss: 0.95545
update_step:  50 model loss: 3.00100, kl_loss: 3.00000, obs_loss: 0.00100, reward_loss: 0.00000, value_loss: 0.00002 action_loss: 0.95501
elasped time for update: 54.27s
episode [  70/1000] is collected. Total reward is -0.999200
elasped time for interaction: 1.37s
update_step:  10 model loss: 3.00095, kl_loss: 3.00000, obs_loss: 0.00095, reward_loss: 0.00000, value_loss: 0.00002 action_loss: 0.95458
update_step:  20 model loss: 3.00093, kl_loss: 3.00000, obs_loss: 0.00093, reward_loss: 0.00000, value_loss: 0.00002 action_loss: 0.95422
update_step:  30 model loss: 3.00093, kl_loss: 3.00000, obs_loss: 0.00093, reward_loss: 0.00000, value_loss: 0.00002 action_loss: 0

In [ ]:
competition_folder = "./configs/competition/"
configuration_files = os.listdir(competition_folder)
configuration_random = random.randint(0, len(configuration_files))
configuration_file = (
    competition_folder + configuration_files[configuration_random]
)

aai_env = AnimalAIEnvironment(
        seed = 123,
        file_name="./env/AnimalAI",
        arenas_configurations=configuration_file,
        play=False,
        base_port=5000,
        inference=False,
        useCamera=True,
        resolution=36,
        useRayCasts=True,
        no_graphics=True,
        # raysPerSide=1,
        # rayMaxDegrees = 30,
        worker_id = 3,
    )

# env = UnityToGymWrapper(aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=False)
# def make_env():
#     def _thunk():
#         env = UnityToGymWrapper(aai_env, uint8_visual=False, allow_multiple_obs=True, flatten_branched=True)
#         return env
#     return _thunk
# env = DummyVecEnv([make_env()])
runname = "dqn_baseline"
log_dir = "./logs/" + runname
env = UnityToGymWrapper(aai_env, uint8_visual=True, allow_multiple_obs=False, flatten_branched=True)
env = Monitor(env, log_dir)

policy_kwargs = dict(activation_fn=th.nn.ReLU)
model = DQN("CnnPolicy", env, policy_kwargs=policy_kwargs, verbose=1, tensorboard_log=("./tensorboard/" + runname))

no_saves = 100
no_steps = 1000000
reset_num_timesteps = True
for i in range(no_saves):
    model.learn(no_steps, reset_num_timesteps=reset_num_timesteps)
    model.save("results/" + runname + "/model_" + str( (i+1)*no_steps ))
    reset_num_timesteps = False
env.close()

[INFO] Connected to Unity environment with package version 2.1.0-exp.1 and communication version 1.5.0
[INFO] Connected new brain: AnimalAI?team=0
[WARNING] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.7/dist-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Logging to ./tensorboard/dqn_baseline/DQN_2
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 776      |
|    ep_rew_mean      | -1.18    |
|    exploration_rate | 0.971    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 180      |
|    time_elapsed     | 17       |
|    total_timesteps  | 3105     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 802      |
|    ep_rew_mean      | -1.21    |
|    exploration_rate | 0.939    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 178      |
|    time_elapsed     | 35       |
|    total_timesteps  | 6417     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 804      |
|    ep_rew_mean      | -1.22    |
|    exploration_rate | 0.908    |
| time/    

KeyboardInterrupt: 

In [ ]:
env.close()

NameError: name 'env' is not defined